## 1-minute introduction to Jupyter ##

A Jupyter notebook consists of cells. Each cell contains either text or code.

A text cell will not have any text to the left of the cell. A code cell has `In [ ]:` to the left of the cell.

If the cell contains code, you can edit it. Press <kbd>Enter</kbd> to edit the selected cell. While editing the code, press <kbd>Enter</kbd> to create a new line, or <kbd>Shift</kbd>+<kbd>Enter</kbd> to run the code. If you are not editing the code, select a cell and press <kbd>Ctrl</kbd>+<kbd>Enter</kbd> to run the code.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

# Lesson 12b: Module planning and design

In [ ]:
# This code cell loads a PEP8 linter
# Linting is the process of flagging programming errors,
# bugs, stylistic errors, and other code problems
# pycodestyle is a linter that highlights any syntax that
# is not PEP8-compliant.

%load_ext pycodestyle_magic
%pycodestyle_on

Now that we have our `Board` and pieces defined, we can start to think through what else we need to have a working game.

Before we start to write the code, again, let’s think through what our program needs to do.

**Tasks to be done**

1. Display the game board
2. Ask white player for a move
3. Check if it is a valid move
   - move must follow movement rules of the piece
   - destination must not have a piece of the same colour
4. Update board positions
5. Repeat steps 1–4 for black player

## Task 1: Display the game board

Let’s keep it simple. For now, just display each board position as two ASCII characters, with a two-space gap between positions.

The chess pieces are represented by two letters that indicate their colour and name.
- White pieces are represented as W
- Black pieces are represented as B
- Chess pieces are represented by the first letter of their name (uppercase)

So a white king will be represented as `WK`, a black pawn will be represented as `BP`.

### Expected output

The entire board will look like this:

    BR  BK  BB  BQ  BK  BB  BK  BR

    BP  BP  BP  BP  BP  BP  BP  BP









    WP  WP  WP  WP  WP  WP  WP  WP

    WR  WK  WB  WQ  WK  WB  WK  WR

In [ ]:
from chess import Board, King, Queen, Bishop, Knight, Rook, Pawn


# Define a method, display(), to display the contents of the board
def display(self):
    '''
    Displays the contents of the board.
    Each piece is represented by two letters.
    First letter is the colour (W for white, B for black).
    Second letter is the name (Starting letter for each piece).
    '''
    # Write your code here
    


# Add this method to `Board`
Board.display = display

# Instantiate a board and display it
b = Board()
b.display()

## Task 2: Ask white player for a move

The simplest way for white to move is to indicate the starting position and the ending position. From that input, we can tell which piece is at the starting position, and attempt to move it to the ending position.

Looking ahead, we will need to reuse this for the black player as well. So this method should allow reuse by letting us choose if it should prompt the white player or black player.

If the player gives an invalid input, naturally we should prompt them again to give valid input. It is also considerate to give a useful help message, either informing them of their error, or informing them of the correct value or format needed.

### Expected output

So that would look like this:

    >>> start,end = b.prompt('white')
    White player: 01,09
    Invalid move. Please enter your move in the following format: __ __, _ represents a digit.
    White player: 01 09
    Invalid move. Move digits should be < 8.
    White player: 01 02
    >>> start,end
    ((0,1), (0,2))

### Validation

Naturally, we need to validate this input first. What rules do you think are needed for this input?

### Separation of concerns

Code is much easier to read, understand, and debug if we write them so that each chunk of code serves one main purpose. You do yourself and others a service if you write your code so that it validates the data first, and only parses the user input when it has been validated.

Parsing is the act of resolving a line into different parts. In this case, we are parsing a single line of user input into a start position and end position. This task is made much easier if you can assume that the data is validated (obeys certain assumptions).

In [ ]:
def prompt(self, colour):
    # Write code to prompt the user for a move
    # Validate the move input
    # Return the start and end positions
    


# Add this method to `Board`
Board.prompt = prompt
b = Board()
start, end = b.prompt('white')
start, end

## Task 3: Validate move

This task is rather more complex than the other two. Before we write any code, let’s think it through.

We need to:

1. Identify which piece is at the start position
2. Identify the colour and name of the piece
3. Check if the move is valid for that piece name (e.g. a rook cannot move diagonally)
4. Check if there is a piece of the same colour at the end position

(1) and (2) are simple enough. (3) looks complex though. We should dig deeper.

How would we validate the move for each piece?

It makes sense that the rules for movement should be associated with the piece, not the board. We could write it as a method _belonging to that piece’s class_, rather than belonging to `Board`. We could pass the start and end positions as 2-ple (tuple with 2 elements) arguments to this method, and it will return `True` (for valid move) and `False` (for invalid move):

    >>> Pawn.isvalid((0,1),(0,2))
    True
    
We will need to write separate validators for each piece class. You can refer to [this page](http://www.chesscoachonline.com/chess-articles/chess-rules) for basic movement rules (ignore special rules e.g. rook castling)

At this point, you may assume that the data has been validated and does not need further validation. If you encounter any instances of invalid input, you should check your code for `prompt()` in **Task 2**.

Isn’t separation of concerns so lovely?

In [ ]:
# Let's try writing some code for two pieces: Pawn and King


# move validation for Pawn
def isvalid(self, start, end):
    x_move = end[0] - start[0]
    y_move = end[1] - start[1]
    dist = abs(x_move) + abs(y_move)
    if x_move == 0:
        if (self.colour == 'black' and y_move == -1) or \
           (self.colour == 'white' and y_move == 1):
            return True
    else:
        return False


# move validation for King
def isvalid(self, start, end):
    x_move = end[0] - start[0]
    y_move = end[1] - start[1]
    dist = abs(x_move) + abs(y_move)
    if dist == 1:
        return True
    else:
        return False

I can foresee that this chunk of code is going to be needed a lot for the rest of the classes:

    x_move = end[0] - start[0]
    y_move = end[1] - start[1]
    dist = abs(x_move) + abs(y_move)

So much repetition! Let's make a helper function:

In [ ]:
def vector(start, end):
    x_move = end[0] - start[0]
    y_move = end[1] - start[1]
    dist = abs(x_move) + abs(y_move)
    return x_move, y_move, dist

Now we can use `vector()` to help us with the move validation:

In [ ]:
# move validation for Pawn
def isvalid(self, start, end):
    x_move, y_move, dist = vector(start, end)
    if x_move == 0:
        if self.colour == 'black' and y_move == -1:
            return True
        elif self.colour == 'white' and y_move == 1:
            return True
    return False
    # The pawn has a special 'capturing' move.
    # We will handle this separately, later on.


# Add this method to `Pawn`
Pawn.isvalid = isvalid


# move validation for King
def isvalid(self, start, end):
    x_move, y_move, dist = vector(start, end)
    if dist == 1:
        return True
    else:
        return False


# Add this method to `King`
King.isvalid = isvalid

In [ ]:
# Autograder tests for King move validation

testdata = {King('white'): [(4, 0), (4, 1), True],
            King('white'): [(4, 0), (4, 2), False],
            }
for piece, (start, end, ans) in testdata.items():
    result = piece.isvalid(start, end)
    assert result == ans, \
        f'{piece}({start},{end} should return {ans}, got {result} instead)'

Your turn.

### Part 1

Define the move validation methods for the remaining pieces:

In [ ]:
# move validation for Queen
def isvalid(self, start, end):
    x_move, y_move, dist = vector(start, end)
    # Write your move validation here
    


# Add this method to `Queen`
Queen.isvalid = isvalid

In [ ]:
# Autograder tests for Queen move validation

testdata = {Queen('white'): [(3, 0), (5, 2), True],
            Queen('white'): [(3, 0), (5, 3), False],
            }
for piece, (start, end, ans) in testdata.items():
    result = piece.isvalid(start, end)
    assert result == ans, \
        f'{piece}({start},{end} should return {ans}, got {result} instead)'

In [ ]:
# move validation for Bishop
def isvalid(self, start, end):
    x_move, y_move, dist = vector(start, end)
    # Write your move validation here
    


# Add this method to `Bishop`
Bishop.isvalid = isvalid

In [ ]:
# Autograder tests for Bishop move validation

testdata = {Bishop('white'): [(2, 0), (0, 2), True],
            Bishop('white'): [(2, 0), (1, 2), False],
            }
for piece, (start, end, ans) in testdata.items():
    result = piece.isvalid(start, end)
    assert result == ans, \
        f'{piece}({start},{end} should return {ans}, got {result} instead)'

In [ ]:
# move validation for Knight
def isvalid(self, start, end):
    x_move, y_move, dist = vector(start, end)
    # Write your move validation here
    


# Add this method to `Knight`
Knight.isvalid = isvalid

In [ ]:
# Autograder tests for Knight move validation

testdata = {Knight('white'): [(1, 0), (2, 2), True],
            Knight('white'): [(1, 0), (1, 2), False],
            }
for piece, (start, end, ans) in testdata.items():
    result = piece.isvalid(start, end)
    assert result == ans, \
        f'{piece}({start},{end} should return {ans}, got {result} instead)'

In [ ]:
# move validation for Rook
def isvalid(self, start, end):
    x_move, y_move, dist = vector(start, end)
    # Write your move validation here
    


# Add this method to `Rook`
Rook.isvalid = isvalid

In [ ]:
# Autograder tests for Rook move validation

testdata = {Rook('white'): [(0, 0), (0, 1), True],
            Rook('white'): [(0, 0), (1, 1), False],
            }
for piece, (start, end, ans) in testdata.items():
    result = piece.isvalid(start, end)
    assert result == ans, \
        f'{piece}({start},{end} should return {ans}, got {result} instead)'

### Part 2

We have not done step 4 yet: Check if there is a piece of the same colour at the end position.

Thi is something that the piece objects will not be able to do, since they don’t have access to the board information. So we will need to define this method on `Board` instead.

A quick refresher on the `field` attribute that `Board` has:

In [ ]:
b = Board()
b.field

Let’s start by defining a method, `piece_at()` to get information about the piece at a position (`x`,`y`):

In [ ]:
def piece_at(self, coord):
    '''
    Retrieves the piece at `coord`.
    `coord` is assumed to be a 2-ple of ints representing
    (col,row).

    Return:
    dict
    - 'piece': {King, Queen, Bishop, Knight, Rook, Pawn}
    - position: (col,row)
    or None if no piece found
    '''
    for each in self.field:
        if each['position'] == coord:
            return each
    return None


# Add this method to `Board`
Board.piece_at = piece_at

Your turn.

Define the move validation method for `Board`, using the method `.piece_at()` to help you retrieve the pieces from the board:

In [ ]:
def isvalid(self, start, end):
    '''
    Determines if the board move is valid by:
    1. Checking if the piece at `start` and the piece at `end`
       are the same colour

    Returns:
    True if valid
    False if invalid
    '''
    x_move, y_move, dist = vector(start, end)
    start_piece = self.piece_at(start)['piece']
    # Write your code here
    


# Add this method to `Board`
Board.isvalid = isvalid

## Task 4: Update board positions

Once we have confirmed that the move is valid, we can move the piece and update the result. This involves a few steps:

1. Remove any opponent piece that is at that position
2. Update the position of the piece
3. Print the result of the move
   - 'white king 30 -> 31'
   - 'black queen 37 -> 31 captures white pawn'
   
The piece positions are stored in `Board`, and the pieces have no info about their current position, so it makes sense to define this method under `Board` (after all, in real chess, the pieces are moved; they do not move themselves!).

A quick refresher on the attributes and methods of a chess piece:

In [ ]:
k = King('white')
for attr in dir(k):
    if not attr.startswith('__'):
        print(attr)

Define a method to move a piece from `start` position to `end` position.

- You may assume that `start` and `end` are 2-ples containing a pair of `int`s. The first `int` is the column index, the second `int` is the row index.
- You may assume the move has been validated.

### Part 1

Define an `index_at()` method that:

1. Takes in a `list` of 2-ple positions,
2. Returns a `list` of `int`s representing the `self.field` indexes of the pieces at those positions.
   - return `None` if there is no piece at that position.

**Hint 1:** The code to search for the piece at the `start` position is very similar to `piece_at()` method, so you can modify your code from that.

In [ ]:
def index_at(self, pos_list):
    '''
    Get the indexes of board pieces at the positions
    given in `pos_list`.
    Each position is a 2-ple represent col,row.

    Return:
    tuple of `int` corresponding to piece index,
    None if no piece at that position
    '''
    # Write your code here
    


# Add this method to `Board`
Board.index_at = index_at

In [ ]:
# TEST: Check correct output for `index_at()`
test_b = Board()

# Get all the piece positions
test_pos_list = [test_b.field[i]['position'] for i in range(len(test_b.field))]
# Return all piece indexes
result_index_list = test_b.index_at(test_pos_list)

# Test that position returned by board matches returned list
for pos_index, piece_index in enumerate(result_index_list):
    piece = test_b.field[piece_index]['piece']
    position = test_b.field[piece_index]['position']
    result = test_pos_list[pos_index]
    assert position == result, \
        f'Piece index {piece_index} ({str(piece)}) ' \
        f'is at {position}, got {result} instead.'
# Printed if no AssertionError raised
print('All OK')

### Part 2

Update the code in the `move_piece()` method to

1. Remove the `self.field` dict at `end` position index (if any),
2. Set the `'position'` key of `self.field` dict at `start` position index to the `end` position.


In [ ]:
def move_piece(self, start, end):
    '''
    Move piece from `start` to `end`.

    1. Removes any opponent pieces at `end`,
    2. Updates the position of `piece` in the board.

    Return:
    opponent_piece if opponent piece removed
    None if no opponent piece removed
    '''
    # Write your code here
    


# Add this method to `Board`
Board.move_piece = move_piece

In [ ]:
# TEST: player_piece successfully moved, return value is None

# Move black rook
start = (7, 7)
end = (7, 4)

test_b = Board()
piece_index = test_b.index_at([start])[0]
opponent_piece = test_b.move_piece(start, end)
own_piece = test_b.field[piece_index]['piece']
own_piece_pos = test_b.field[piece_index]['position']

assert own_piece_pos == end, \
    f'{own_piece} {start} -> {end}, result was {own_piece_pos} instead.'
assert opponent_piece is None, \
    'Return value is not None (no opponent piece removed)'
# Printed if no AssertionError raised
print('All OK')

In [ ]:
# TEST: opponent_piece successfully removed from board,
#       returned by move_piece()

# Black queen takes white pawn
start = (3, 7)
end = (3, 1)

test_b = Board()
piece_index = test_b.index_at([start])[0]
opponent_piece = test_b.move_piece(start, end)['piece']
own_piece = test_b.field[piece_index]['piece']
own_piece_pos = test_b.field[piece_index]['position']

assert own_piece_pos == end, \
    f'{own_piece} {start} -> {end}, result was {own_piece_pos} instead.'
assert str(opponent_piece) == 'white pawn', \
    f'white pawn not taken (opponent piece was {opponent_piece})'
# Printed if no AssertionError raised
print('All OK')

## Polymorphism

One thing that made this game so easy to write is that all the piece classes use the same method names. Besides the methods they inherited from `BasePiece`, each piece also has an `isvalid()` method. Although the `isvalid()` method for each piece works differently, it takes in the same type of input and returns the same type of output.

As long as we can _assume_ that every piece has an `isvalid()` method that will return a `bool`, we dont need to worry about the _details_ of how the piece class is implemented. This feature of being able to use different classes with the same code is known as **polymorphism**.

## Endgame

We now have the basic classes and methods in place to write the main code of the game. See you in Assignment 12.

# Feedback and suggestions

Any feedback or suggestions for this assignment?

YOUR ANSWER HERE